# Synthesys 3 Query Examples
*Note, this notebook was built with Python 3.6, many of the queries won't work for earlier versions of Python*

This notebook shows how to build the queries to the SYNTHESYS datasets, using Carto's MAP API and SQL API. We will use the data in three basic ways: 1) to build choropleths, 2) to build widgets, and 3) to build dynamic sentences.

We have created an account on Carto (`synthesys`), which contains several data tables.
1. sanitized_data : A the access dataset from SYNTHESYS. It includes geocoded points.
1. gadm28_adm1 : the GADM28 admin 1 polygons. ISO codes are in 3 character format (var:iso)
1. gadm28_countries  : GADM28 country polygons. ISO codes are in 2 character format (var:iso2)

We will need to create a variety of widgets from these data, including choropleth maps, donut plots, bubble plots, treeplots, and time series bar charts. There is also a Sankey diagram to produce, which is a bit more complex. It is described at the end of this notebook.

We will also use the sql api to retrieve data for dynamic text (a sentence that should include variables) detailed below.


## Filters and Metadata

When a user hits the dashboard page, they will first see an unfiltered view of the data. From this page they will be able to apply three filters: 1) Country, 2) Discipline, and 3) Funding round.

#### Country

The below JSON structure gives the ISO2 code paired with the English country name. It can be used on the front end to build the filter. A user can select the english country name, and the corresponding ISO2 code can be sent as a variable in the SQL queries given on this page to return data for that country from Carto.

Note the ISO2 codes are used to query the `gadm28_countries` table. Whereas ISO3 codes are used for the `gadm28_adm1` table (and should be used for SQL requests being applied with the COUNTRY FILTER).

*ISO2 codes*
```json
{'AL': 'Albania',
 'AT': 'Austria',
 'BA': 'Bosnia and Herzegovina',
 'BE': 'Belgium',
 'BG': 'Bulgaria',
 'CH': 'Switzerland',
 'CY': 'Cyprus',
 'CZ': 'Czech Republic',
 'DE': 'Germany',
 'DK': 'Denmark',
 'EE': 'Estonia',
 'ES': 'Spain',
 'FI': 'Finland',
 'FR': 'France',
 'GB': 'United Kingdom',
 'GF': 'French Guiana',
 'GR': 'Greece',
 'HR': 'Croatia',
 'HU': 'Hungary',
 'IE': 'Ireland',
 'IL': 'Israel',
 'IS': 'Iceland',
 'IT': 'Italy',
 'LI': 'Liechtenstein',
 'LT': 'Lithuania',
 'LU': 'Luxembourg',
 'LV': 'Latvia',
 'MK': 'Macedonia',
 'MT': 'Malta',
 'NL': 'Netherlands',
 'NO': 'Norway',
 'PL': 'Poland',
 'PS': 'Palestina',
 'PT': 'Portugal',
 'RE': 'Reunion',
 'RO': 'Romania',
 'RS': 'Serbia',
 'SE': 'Sweden',
 'SI': 'Slovenia',
 'SK': 'Slovakia',
 'TR': 'Turkey'}
```

*ISO3 codes*

```json
{'ALB': 'Albania',
 'AUT': 'Austria',
 'BEL': 'Belgium',
 'BGR': 'Bulgaria',
 'BIH': 'Bosnia and Herzegovina',
 'CHE': 'Switzerland',
 'CYP': 'Cyprus',
 'CZE': 'Czech Republic',
 'DEU': 'Germany',
 'DNK': 'Denmark',
 'ESP': 'Spain',
 'EST': 'Estonia',
 'FIN': 'Finland',
 'FRA': 'France',
 'GBR': 'United Kingdom',
 'GRC': 'Greece',
 'GUF': 'French Guiana',
 'HRV': 'Croatia',
 'HUN': 'Hungary',
 'IRL': 'Ireland',
 'ISL': 'Iceland',
 'ISR': 'Israel',
 'ITA': 'Italy',
 'LIE': 'Liechtenstein',
 'LTU': 'Lithuania',
 'LUX': 'Luxembourg',
 'LVA': 'Latvia',
 'MKD': 'Macedonia',
 'NLD': 'Netherlands',
 'NOR': 'Norway',
 'POL': 'Poland',
 'PRT': 'Portugal',
 'PSE': 'Palestina',
 'REU': 'Reunion',
 'ROU': 'Romania',
 'SRB': 'Serbia',
 'SVK': 'Slovakia',
 'SVN': 'Slovenia',
 'SWE': 'Sweden',
 'TUR': 'Turkey'}
 ```

#### Discipline

```json
{'discipline': ['Earth Sciences & Environment',
                'Engineering & Technology',
                'Humanities',
                'Information & Communication Technologies',
                'Life Sciences & Biotech',
                'Material Sciences',
                'Physics',
                'Social Sciences']}
```

#### Funding round

Users should be able to filter the data by funding round (attribute `synth_roun`): there are three possible values of this, corresponding to three funding rounds.

```json
{'synth_roun': ['R1', 'R2', 'R3']}
```

The data in the SQL table are refrenced by the above codes, however, they should be exposed to the user as the following (including in all figures and instances). I.e. the user should see the below mapping and not R1 etc.
```json
{"R1":"SYNTHESYS (2004 - 2009)",
 "R2":"SYNTHESYS2 (2009 - 2013)",
 "R3":"SYNTHESYS3 (2013 - 2017)"}
```

#### Institute visited
The access data shows visits by a person to an Institute. The institutes have a unique ID code, given in the table.attribute `sanitized_data.inst_id`. The mapping between the `inst_id` and a human-readable name is given in the JSON below, and should be used to expose these data to the user.(I.e. the user should see the below human readable text as labels, rather than the `inst_id`. (This info will be used to create the treemap widget.)

```json
{0: 'Botanischer Garten und Botanisches Museum Collections',
 1: 'Botanischer Garten und Botanisches Museum Herbarium',
 2: 'Botanischer Garten und Botanisches Museum Laboratories',
 3: 'Botanischer Garten und Botanisches Museum Living Collections',
 4: 'Botanischer Garten und Botanisches Museum Molecular Laboratory',
 5: 'Botanischer Garten und Botanisches Museum Scanning Electron Microscope facilities',
 6: 'Consejo Superior de Investigaciones Científicas',
 7: 'Hungarian Natural History Museum',
 8: 'Hungarian Natural History Museum Collections',
 9: 'Hungarian Natural History Museum Computing facilities',
 10: 'Hungarian Natural History Museum Microscopy facilities',
 11: 'Hungarian Natural History Museum Molecular Analysis and Microscopy facilities',
 12: 'Museum National d’Histoire Naturelle Analytical Facilities',
 13: 'Museum National d’Histoire Naturelle Collections',
 14: 'Museum National d’Histoire Naturelle Facilities',
 15: 'Museum National d’Histoire Naturelle Mass Spectrometry facilities',
 16: 'Museum National d’Histoire Naturelle Molecular Laboratory',
 17: 'Museum National d’Histoire Naturelle Scanning Electron Microscope facilities',
 18: 'Museum and Institute of Zoology of the Polish Academy of Sciences',
 19: 'Museum für Naturkunde Collections',
 20: 'Museum für Naturkunde Laboratories',
 21: 'Museum für Naturkunde Mineralogy',
 22: 'Museum für Naturkunde Palaeontology',
 23: 'Museum für Naturkunde Zoology',
 24: 'National Museum Prague',
 25: 'Natural History Museum London Analytical Facilities',
 26: 'Natural History Museum London Collections',
 27: 'Natural History Museum London Collections and Laboratories',
 28: 'Natural History Museum London Laboratories',
 29: 'Natural History Museum Vienna',
 30: 'Naturalis Biodiversity Center',
 31: 'Naturhistoriska riksmuseet',
 32: 'Royal Belgian Institute of Natural Sciences',
 33: 'Royal Botanic Garden Edinburgh',
 34: 'Royal Botanic Gardens Kew',
 35: 'Royal Museum of Central Africa',
 36: 'Senckenberg Gesellschaft für Naturforschung Collections',
 37: 'Senckenberg Gesellschaft für Naturforschung Laboratories',
 38: 'State Museum of Natural History Stuttgart',
 39: 'University of Amsterdam',
 40: 'University of Copenhagen'}
```

## Pre-calculated values

In some cases, there isn't a need to query the datasets via the backend, as the json is lightweight and needed repeatedly, it can be embedded in the site:

For example, the following table gives *the total number of research visits by country* for all times and disciplines. 

Please use your own discretion to decide when is best to embed the data into the site, and when is best to call the data dynamically.

```json
{'fields': {'count': {'type': 'number'},
            'iso': {'type': 'string'},
            'name_engli': {'type': 'string'}},
 'rows': [{'count': 903, 'iso': 'DE', 'name_engli': 'Germany'},
          {'count': 599, 'iso': 'GB', 'name_engli': 'United Kingdom'},
          {'count': 521, 'iso': 'IT', 'name_engli': 'Italy'},
          {'count': 500, 'iso': 'ES', 'name_engli': 'Spain'},
          {'count': 460, 'iso': 'PL', 'name_engli': 'Poland'},
          {'count': 360, 'iso': 'CZ', 'name_engli': 'Czech Republic'},
          {'count': 338, 'iso': 'FR', 'name_engli': 'France'},
          {'count': 202, 'iso': 'PT', 'name_engli': 'Portugal'},
          {'count': 164, 'iso': 'BG', 'name_engli': 'Bulgaria'},
          {'count': 159, 'iso': 'BE', 'name_engli': 'Belgium'},
          {'count': 147, 'iso': 'AT', 'name_engli': 'Austria'},
          {'count': 143, 'iso': 'HU', 'name_engli': 'Hungary'},
          {'count': 134, 'iso': 'NL', 'name_engli': 'Netherlands'},
          {'count': 102, 'iso': 'TR', 'name_engli': 'Turkey'},
          {'count': 98, 'iso': 'SE', 'name_engli': 'Sweden'},
          {'count': 92, 'iso': 'CH', 'name_engli': 'Switzerland'},
          {'count': 87, 'iso': 'RO', 'name_engli': 'Romania'},
          {'count': 80, 'iso': 'FI', 'name_engli': 'Finland'},
          {'count': 73, 'iso': 'DK', 'name_engli': 'Denmark'},
          {'count': 67, 'iso': 'IE', 'name_engli': 'Ireland'},
          {'count': 65, 'iso': 'SK', 'name_engli': 'Slovakia'},
          {'count': 64, 'iso': 'NO', 'name_engli': 'Norway'},
          {'count': 62, 'iso': 'GR', 'name_engli': 'Greece'},
          {'count': 57, 'iso': 'RS', 'name_engli': 'Serbia'},
          {'count': 52, 'iso': 'IL', 'name_engli': 'Israel'},
          {'count': 35, 'iso': 'EE', 'name_engli': 'Estonia'},
          {'count': 33, 'iso': 'HR', 'name_engli': 'Croatia'},
          {'count': 27, 'iso': 'LT', 'name_engli': 'Lithuania'},
          {'count': 20, 'iso': 'MK', 'name_engli': 'Macedonia'},
          {'count': 16, 'iso': 'PS', 'name_engli': 'Palestina'},
          {'count': 10, 'iso': 'SI', 'name_engli': 'Slovenia'},
          {'count': 7, 'iso': 'IS', 'name_engli': 'Iceland'},
          {'count': 7, 'iso': 'LV', 'name_engli': 'Latvia'},
          {'count': 6, 'iso': 'AL', 'name_engli': 'Albania'},
          {'count': 5, 'iso': 'GF', 'name_engli': 'French Guiana'},
          {'count': 4, 'iso': 'BA', 'name_engli': 'Bosnia and Herzegovina'},
          {'count': 4, 'iso': 'RE', 'name_engli': 'Reunion'},
          {'count': 2, 'iso': 'MT', 'name_engli': 'Malta'},
          {'count': 1, 'iso': 'CY', 'name_engli': 'Cyprus'},
          {'count': 1, 'iso': 'LI', 'name_engli': 'Liechtenstein'},
          {'count': 1, 'iso': 'LU', 'name_engli': 'Luxembourg'}],
 'time': 12.721,
 'total_rows': 41}
 ```

## Admin 1 level queries

The following code shows how to create choropleth plots for ADMIN 1 level data.

#### Filter: By Country

If a user is filtering by country, they should see a choropleth that shows ADMIN 1 level view of that country.

To create a choropleth of ADMIN 1 areas, for a single country, use the following SQL query to the CARTO MAP API. Note the `{variable_iso}` should be whatever three character country code we wish to set (e.g. 'DEU' for Germany).
A json mapping of the country names to the three and two letter iso codes is given in this notebook.

```sql
     WITH gadm28 as (select the_geom_webmercator FROM gadm28_adm1 WHERE iso={variable_iso})
           SELECT gadm28.the_geom_webmercator, count(synthesys.home_insti)
           FROM  sanitized_data as synthesys, gadm28
           WHERE ST_Intersects(gadm28.the_geom_webmercator, synthesys.the_geom_webmercator)
           GROUP BY gadm28.the_geom_webmercator
```


These codes should come from a drop-down filter menu. Valid possibilities for the filters are listed in this notebook.

In [1]:
import folium
import pprint
import requests
import json

In [2]:
# example of getting admin1 choropleth for Germany

variable_iso = 'DEU'

query = f"""with gadm28 as (select the_geom_webmercator from  gadm28_adm1 where iso='{variable_iso}')
           SELECT gadm28.the_geom_webmercator, count(synthesys.home_insti)
           FROM  sanitized_data as synthesys, gadm28
           where ST_Intersects(  gadm28.the_geom_webmercator, synthesys.the_geom_webmercator)
           group by gadm28.the_geom_webmercator"""

style = """#layer {polygon-fill: ramp([count],
                   colorbrewer(Reds), jenks());
                   polygon-opacity: 0.9;}"""


account = 'synthesys'
urlCarto = 'https://'+account+'.carto.com/api/v1/map'
body = {
    "layers": [{
        "type": "cartodb",
        "options": {
            "sql": query,
            "cartocss":style,
            "cartocss_version": "2.1.1"
        }
    }]
}


r = requests.post(urlCarto, data=json.dumps(body), headers={'content-type': 'application/json; charset=UTF-8'})
print(r.url)
pprint.pprint(r.json())

tileUrl = 'https://'+account+'.carto.com/api/v1/map/' + r.json()['layergroupid'] + '/{z}/{x}/{y}.png32';

map_osm = folium.Map(location=[45.5236, 0.6750], zoom_start=3)
folium.TileLayer(
    tiles=tileUrl,
    attr='text',
    name='text',
    overlay=True
).add_to(map_osm)
map_osm

https://synthesys.carto.com/api/v1/map
{'cdn_url': {'http': 'ashbu.cartocdn.com',
             'https': 'cartocdn-ashbu.global.ssl.fastly.net',
             'templates': {'http': {'subdomains': ['0', '1', '2', '3'],
                                    'url': 'http://{s}.ashbu.cartocdn.com'},
                           'https': {'subdomains': ['a', 'b', 'c', 'd'],
                                     'url': 'https://cartocdn-ashbu_{s}.global.ssl.fastly.net'}}},
 'last_updated': '2017-07-28T13:44:23.833Z',
 'layergroupid': 'af483349fc3a2df337971dfd547775da:1501249463833',
 'metadata': {'analyses': [],
              'dataviews': {},
              'layers': [{'id': 'layer0',
                          'meta': {'cartocss': '#layer {polygon-fill: '
                                               '#fee5d9;[ count > 22 ] '
                                               '{polygon-fill: #fcae91;}[ '
                                               'count > 47 ] {polygon-fill: '
                     

If data are required from the map/or we need an accompanying histogram of the values from each area:

```sql
with gadm28 as (SELECT the_geom_webmercator, iso, name_1 FROM  gadm28_adm1 WHERE iso='DEU')
 SELECT gadm28.iso, gadm28.name_1, COUNT(synthesys.home_insti)
 FROM  sanitized_data as synthesys, gadm28
 where ST_Intersects(  gadm28.the_geom_webmercator, synthesys.the_geom_webmercator)
 group by gadm28.the_geom_webmercator, gadm28.iso, gadm28.name_1
 ORDER BY count DESC```

In [56]:
# example of getting the data of the choropleth to make a histogram for the german choropleth if needed

variable_iso = 'DEU'

query = f"""with gadm28 as (SELECT the_geom_webmercator, iso, name_1 FROM  gadm28_adm1 WHERE iso='{variable_iso}')
 SELECT gadm28.iso, gadm28.name_1, COUNT(synthesys.home_insti)
 FROM  sanitized_data as synthesys, gadm28
 where ST_Intersects(  gadm28.the_geom_webmercator, synthesys.the_geom_webmercator)
 group by gadm28.iso, gadm28.name_1
 ORDER BY count DESC"""

account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"

sql = {"q": query}
r = requests.get(urlCarto, params=sql)
print(r.url)
#pprint.pprint(r.json())

https://synthesys.carto.com/api/v2/sql?q=with+gadm28+as+%28SELECT+the_geom_webmercator%2C+iso%2C+name_1+FROM++gadm28_adm1+WHERE+iso%3D%27DEU%27%29%0A+SELECT+gadm28.iso%2C+gadm28.name_1%2C+COUNT%28synthesys.home_insti%29%0A+FROM++sanitized_data+as+synthesys%2C+gadm28%0A+where+ST_Intersects%28++gadm28.the_geom_webmercator%2C+synthesys.the_geom_webmercator%29%0A+group+by+gadm28.iso%2C+gadm28.name_1%0A+ORDER+BY+count+DESC


The above request should return the following json data, which you can use to build a plot:

```json
{'fields': {'count': {'type': 'number'},
            'iso': {'type': 'string'},
            'name_1': {'type': 'string'}},
 'rows': [{'count': 188, 'iso': 'DEU', 'name_1': 'Bayern'},
          {'count': 140, 'iso': 'DEU', 'name_1': 'Baden-Württemberg'},
          {'count': 116, 'iso': 'DEU', 'name_1': 'Hessen'},
          {'count': 94, 'iso': 'DEU', 'name_1': 'Nordrhein-Westfalen'},
          {'count': 79, 'iso': 'DEU', 'name_1': 'Niedersachsen'},
          {'count': 70, 'iso': 'DEU', 'name_1': 'Berlin'},
          {'count': 47, 'iso': 'DEU', 'name_1': 'Sachsen'},
          {'count': 41, 'iso': 'DEU', 'name_1': 'Hamburg'},
          {'count': 37, 'iso': 'DEU', 'name_1': 'Thüringen'},
          {'count': 22, 'iso': 'DEU', 'name_1': 'Rheinland-Pfalz'},
          {'count': 20, 'iso': 'DEU', 'name_1': 'Mecklenburg-Vorpommern'},
          {'count': 15, 'iso': 'DEU', 'name_1': 'Schleswig-Holstein'},
          {'count': 15, 'iso': 'DEU', 'name_1': 'Bremen'},
          {'count': 10, 'iso': 'DEU', 'name_1': 'Brandenburg'},
          {'count': 9, 'iso': 'DEU', 'name_1': 'Sachsen-Anhalt'}],
 'time': 0.874,
 'total_rows': 15}
```

## Country-wide queries

The Discipline and Funding round filters should trigger Europe-wide choropleths. This should also be displayed by default (without any filters):


* Default: no filter
```sql
WITH gadm28 AS (SELECT the_geom_webmercator, iso2 as iso FROM  gadm28_countries)
 SELECT gadm28.the_geom_webmercator, gadm28.iso, COUNT(synthesys.home_insti) AS count
 FROM  sanitized_data AS synthesys, gadm28
 WHERE ST_Intersects(gadm28.the_geom_webmercator, synthesys.the_geom_webmercator)
 GROUP BY gadm28.the_geom_webmercator, gadm28.iso
```


* By discipline
```sql
WITH gadm28 AS (SELECT the_geom_webmercator, iso2 as iso FROM  gadm28_countries)
 SELECT gadm28.the_geom_webmercator, gadm28.iso, COUNT(synthesys.home_insti) AS count
 FROM  sanitized_data AS synthesys, gadm28
 WHERE ST_Intersects(gadm28.the_geom_webmercator, synthesys.the_geom_webmercator)
 and synthesys.discipline like 'Earth Sciences & Environment'
 GROUP BY gadm28.the_geom_webmercator, gadm28.iso
```

* By funding round
```sql
WITH gadm28 AS (SELECT the_geom_webmercator, iso2 as iso FROM  gadm28_countries)
 SELECT gadm28.the_geom_webmercator, gadm28.iso, COUNT(synthesys.home_insti) AS count
 FROM  sanitized_data AS synthesys, gadm28
 WHERE ST_Intersects(gadm28.the_geom_webmercator, synthesys.the_geom_webmercator)
 AND synthesys.discipline IS NOT null
 AND synthesys.synth_roun = 'R1'
 GROUP BY gadm28.the_geom_webmercator, gadm28.iso
```

In [49]:
discipline_var = 'Earth Sciences & Environment'

query = f"""WITH gadm28 AS (SELECT the_geom_webmercator, iso2 as iso FROM  gadm28_countries)
 SELECT gadm28.the_geom_webmercator, gadm28.iso, COUNT(synthesys.home_insti) AS count
 FROM  sanitized_data AS synthesys, gadm28
 WHERE ST_Intersects(gadm28.the_geom_webmercator, synthesys.the_geom_webmercator)
 and synthesys.discipline like '{discipline_var}'
 GROUP BY gadm28.the_geom_webmercator, gadm28.iso"""


style = """#layer {polygon-fill: ramp([count], colorbrewer(Reds), jenks());
                   polygon-opacity: 0.9;}"""

account = 'synthesys'
urlCarto = 'https://'+account+'.carto.com/api/v1/map'
body = {
    "layers": [{
        "type": "cartodb",
        "options": {
            "sql": query,
            "cartocss":style,
            "cartocss_version": "2.1.1"
        }
    }]
}

r = requests.post(urlCarto, data=json.dumps(body), headers={'content-type': 'application/json; charset=UTF-8'})
print(r.url)
pprint.pprint(r.json())

tileUrl = 'https://'+account+'.carto.com/api/v1/map/' + r.json()['layergroupid'] + '/{z}/{x}/{y}.png32';

map_osm2 = folium.Map(location=[45.5236, 0.6750], zoom_start=3)
folium.TileLayer(
    tiles=tileUrl,
    attr='text',
    name='text',
    overlay=True
).add_to(map_osm2)

map_osm2

https://synthesys.carto.com/api/v1/map
{'cdn_url': {'http': 'ashbu.cartocdn.com',
             'https': 'cartocdn-ashbu.global.ssl.fastly.net',
             'templates': {'http': {'subdomains': ['0', '1', '2', '3'],
                                    'url': 'http://{s}.ashbu.cartocdn.com'},
                           'https': {'subdomains': ['a', 'b', 'c', 'd'],
                                     'url': 'https://cartocdn-ashbu_{s}.global.ssl.fastly.net'}}},
 'last_updated': '2017-07-27T09:10:25.375Z',
 'layergroupid': '1a34bd6fc460b4e56eba432764b6fb04:1501146625375',
 'metadata': {'analyses': [],
              'dataviews': {},
              'layers': [{'id': 'layer0',
                          'meta': {'cartocss': '#layer {polygon-fill: '
                                               '#fee5d9;[ count > 22 ] '
                                               '{polygon-fill: #fcae91;}[ '
                                               'count > 66 ] {polygon-fill: '
                     

We can use similarly structured queries to pull back the data behind the choropleths too, to add as histograms.
e.g. for country counts, with a discipline filter applied:
```sql
WITH gadm28 as (SELECT the_geom_webmercator, iso2 AS iso, name_engli FROM  gadm28_countries)
  SELECT gadm28.iso, gadm28.name_engli, COUNT(synthesys.home_insti)
  FROM  sanitized_data as synthesys, gadm28
  where ST_Intersects(  gadm28.the_geom_webmercator, synthesys.the_geom_webmercator)
  AND synthesys.discipline like 'Earth Sciences & Environment'
  group by gadm28.the_geom_webmercator, gadm28.iso, gadm28.name_engli
  ORDER BY count DESC
```

In [74]:
# Histogram like query: by country

discipline_var = 'Earth Sciences & Environment'

query = f"""WITH gadm28 AS (SELECT the_geom_webmercator, iso2 AS iso, name_engli FROM  gadm28_countries)
             SELECT gadm28.iso, gadm28.name_engli, COUNT(synthesys.home_insti)
             FROM  sanitized_data as synthesys, gadm28
             WHERE ST_Intersects(  gadm28.the_geom_webmercator, synthesys.the_geom_webmercator)
             AND synthesys.discipline like '{discipline_var}'
             GROUP BY gadm28.iso, gadm28.name_engli
             ORDER BY count DESC  """

account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"

sql = {"q": query}
r = requests.get(urlCarto, params=sql)
print(r.url)
#pprint.pprint(r.json())

https://synthesys.carto.com/api/v2/sql?q=WITH+gadm28+AS+%28SELECT+the_geom_webmercator%2C+iso2+AS+iso%2C+name_engli+FROM++gadm28_countries%29%0A+%0A+SELECT+gadm28.iso%2C+gadm28.name_engli%2C+COUNT%28synthesys.home_insti%29%0A+FROM++sanitized_data+as+synthesys%2C+gadm28%0A+WHERE+ST_Intersects%28++gadm28.the_geom_webmercator%2C+synthesys.the_geom_webmercator%29%0A+GROUP+BY+gadm28.iso%2C+gadm28.name_engli%0A+ORDER+BY+count+DESC++


```json
{'fields': {'count': {'type': 'number'},
            'iso': {'type': 'string'},
            'name_engli': {'type': 'string'}},
 'rows': [{'count': 477, 'iso': 'DE', 'name_engli': 'Germany'},
          {'count': 341, 'iso': 'IT', 'name_engli': 'Italy'},
          {'count': 337, 'iso': 'ES', 'name_engli': 'Spain'},
          {'count': 320, 'iso': 'GB', 'name_engli': 'United Kingdom'},
          {'count': 249, 'iso': 'PL', 'name_engli': 'Poland'},
          {'count': 208, 'iso': 'FR', 'name_engli': 'France'},
          {'count': 204, 'iso': 'CZ', 'name_engli': 'Czech Republic'},
          {'count': 143, 'iso': 'PT', 'name_engli': 'Portugal'},
          {'count': 111, 'iso': 'BG', 'name_engli': 'Bulgaria'},
          {'count': 91, 'iso': 'BE', 'name_engli': 'Belgium'},
          {'count': 83, 'iso': 'HU', 'name_engli': 'Hungary'},
          {'count': 78, 'iso': 'AT', 'name_engli': 'Austria'},
          {'count': 69, 'iso': 'NL', 'name_engli': 'Netherlands'},
          {'count': 66, 'iso': 'SE', 'name_engli': 'Sweden'},
          {'count': 55, 'iso': 'RO', 'name_engli': 'Romania'},
          {'count': 52, 'iso': 'GR', 'name_engli': 'Greece'},
          {'count': 39, 'iso': 'IE', 'name_engli': 'Ireland'},
          {'count': 37, 'iso': 'TR', 'name_engli': 'Turkey'},
          {'count': 35, 'iso': 'FI', 'name_engli': 'Finland'},
          {'count': 32, 'iso': 'CH', 'name_engli': 'Switzerland'},
          {'count': 27, 'iso': 'EE', 'name_engli': 'Estonia'},
          {'count': 24, 'iso': 'NO', 'name_engli': 'Norway'},
          {'count': 22, 'iso': 'DK', 'name_engli': 'Denmark'},
          {'count': 22, 'iso': 'IL', 'name_engli': 'Israel'},
          {'count': 21, 'iso': 'HR', 'name_engli': 'Croatia'},
          {'count': 20, 'iso': 'SK', 'name_engli': 'Slovakia'},
          {'count': 18, 'iso': 'RS', 'name_engli': 'Serbia'},
          {'count': 12, 'iso': 'MK', 'name_engli': 'Macedonia'},
          {'count': 11, 'iso': 'LT', 'name_engli': 'Lithuania'},
          {'count': 6, 'iso': 'PS', 'name_engli': 'Palestina'},
          {'count': 6, 'iso': 'LV', 'name_engli': 'Latvia'},
          {'count': 5, 'iso': 'AL', 'name_engli': 'Albania'},
          {'count': 4, 'iso': 'SI', 'name_engli': 'Slovenia'},
          {'count': 3, 'iso': 'BA', 'name_engli': 'Bosnia and Herzegovina'},
          {'count': 3, 'iso': 'RE', 'name_engli': 'Reunion'},
          {'count': 2, 'iso': 'GF', 'name_engli': 'French Guiana'},
          {'count': 2, 'iso': 'IS', 'name_engli': 'Iceland'},
          {'count': 1, 'iso': 'CY', 'name_engli': 'Cyprus'},
          {'count': 1, 'iso': 'LU', 'name_engli': 'Luxembourg'}],
 'time': 16.133,
 'total_rows': 39}
```

## Admin-level 1 across europe

Althouh the designs do not explicitly call for this plot, it will be useful to demonstrate how to make it now, just incase the clients ask for us to add it at a later stage.


In [101]:
query = """WITH gadm28 AS (SELECT the_geom_webmercator, iso FROM  gadm28_adm1)
 SELECT gadm28.the_geom_webmercator, gadm28.iso, COUNT(synthesys.home_insti) AS count
 FROM  sanitized_data AS synthesys, gadm28
 WHERE ST_Intersects(  gadm28.the_geom_webmercator, synthesys.the_geom_webmercator)
 GROUP BY gadm28.the_geom_webmercator, gadm28.iso"""

style = """#layer {polygon-fill: ramp([count], colorbrewer(Reds), jenks());
                   polygon-opacity: 0.9;}"""

account = 'synthesys'
urlCarto = 'https://'+account+'.carto.com/api/v1/map'
body = {
    "layers": [{
        "type": "cartodb",
        "options": {
            "sql": query,
            "cartocss":style,
            "cartocss_version": "2.1.1"
        }
    }]
}

r = requests.post(urlCarto, data=json.dumps(body), headers={'content-type': 'application/json; charset=UTF-8'})
pprint.pprint(r.json())

tileUrl = 'https://'+account+'.carto.com/api/v1/map/' + r.json()['layergroupid'] + '/{z}/{x}/{y}.png32';

map_osm3 = folium.Map(location=[45.5236, 0.6750], zoom_start=3)
folium.TileLayer(
    tiles=tileUrl,
    attr='text',
    name='text',
    overlay=True
).add_to(map_osm3)
map_osm3

{'cdn_url': {'http': 'ashbu.cartocdn.com',
             'https': 'cartocdn-ashbu.global.ssl.fastly.net',
             'templates': {'http': {'subdomains': ['0', '1', '2', '3'],
                                    'url': 'http://{s}.ashbu.cartocdn.com'},
                           'https': {'subdomains': ['a', 'b', 'c', 'd'],
                                     'url': 'https://cartocdn-ashbu_{s}.global.ssl.fastly.net'}}},
 'last_updated': '2017-07-27T09:10:25.375Z',
 'layergroupid': 'd198762fda46afe9615b84f82f6729fb:1501146625375',
 'metadata': {'analyses': [],
              'dataviews': {},
              'layers': [{'id': 'layer0',
                          'meta': {'cartocss': '#layer {polygon-fill: '
                                               '#fee5d9;[ count > 33 ] '
                                               '{polygon-fill: #fcae91;}[ '
                                               'count > 73 ] {polygon-fill: '
                                               '#fb6a4a;}[ c

##  WIDGETS: 
### Examples of querying the SQL API

Note that when the users go to the dashboard page, before they apply a filter they should see the entire (unfiltered) dataset. I have included these queries in the list, but it may be useful to simply grab the json of these data, and embedd it directly in the page, so it loads faster.

In [97]:
account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"

### Donut chart of gender

Again, these data are actually quite lightweight, and if you feel it is a better choice from a development/performance perspective, you could probably make all these queries while building the site, and simply embed the json for all countries into the site. The donut chart should be filtered in four ways:

* No filter - this can probably just be embedded to save the request
* By country
* By discipline
* By Synthysis funding round call

In [123]:
# Men/women donut: no filter
sql = {"q":"SELECT gender, count(*) FROM sanitized_data group by gender"}
account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())

https://synthesys.carto.com/api/v2/sql?q=SELECT+gender%2C+count%28%2A%29+FROM+sanitized_data+group+by+gender
{'fields': {'count': {'type': 'number'}, 'gender': {'type': 'string'}},
 'rows': [{'count': 3722, 'gender': 'M'}, {'count': 1996, 'gender': 'F'}],
 'time': 0.003,
 'total_rows': 2}


In [122]:
# Men/women donut: filter by country
iso2_var = 'ES'
query = f"""WITH gadm28 AS (SELECT the_geom_webmercator, iso2 FROM  gadm28_countries WHERE iso2 = '{iso2_var}')
             SELECT synthesys.gender, count(*)
             FROM  sanitized_data as synthesys, gadm28
             WHERE ST_Intersects(gadm28.the_geom_webmercator, synthesys.the_geom_webmercator)
             GROUP BY synthesys.gender"""
sql = {"q": query}
account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())

https://synthesys.carto.com/api/v2/sql?q=WITH+gadm28+AS+%28SELECT+the_geom_webmercator%2C+iso2+FROM++gadm28_countries+WHERE+iso2+%3D+%27ES%27%29%0A+++++++++++++SELECT+synthesys.gender%2C+count%28%2A%29%0A+++++++++++++FROM++sanitized_data+as+synthesys%2C+gadm28%0A+++++++++++++WHERE+ST_Intersects%28gadm28.the_geom_webmercator%2C+synthesys.the_geom_webmercator%29%0A+++++++++++++GROUP+BY+synthesys.gender
{'fields': {'count': {'type': 'number'}, 'gender': {'type': 'string'}},
 'rows': [{'count': 331, 'gender': 'M'}, {'count': 169, 'gender': 'F'}],
 'time': 0.039,
 'total_rows': 2}


In [121]:
# Men/women donut: filter by discipline
discipline_var = 'Earth Sciences & Environment'
query = f"""SELECT gender, count(synthesys.gender)
             FROM  sanitized_data as synthesys
             WHERE discipline like  '{discipline_var}'
             GROUP BY synthesys.gender"""

account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"
sql = {"q": query}
r = requests.get(urlCarto, params=sql) 
print(r.url)
pprint.pprint(r.json())

https://synthesys.carto.com/api/v2/sql?q=SELECT+gender%2C+count%28synthesys.gender%29%0A+++++++++++++FROM++sanitized_data+as+synthesys%0A+++++++++++++WHERE+discipline+like++%27Earth+Sciences+%26+Environment%27%0A+++++++++++++GROUP+BY+synthesys.gender
{'fields': {'count': {'type': 'number'}, 'gender': {'type': 'string'}},
 'rows': [{'count': 2116, 'gender': 'M'}, {'count': 1122, 'gender': 'F'}],
 'time': 0.005,
 'total_rows': 2}


In [120]:
# Men/Women donut: filter by funding round
funding_round_var = 'R1'
query = """SELECT gender, count(synthesys.gender)
            FROM  sanitized_data as synthesys
            WHERE funding_ro like '{funding_round_var}'
            GROUP BY synthesys.gender"""
account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"
sql = {"q": query}
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())

https://synthesys.carto.com/api/v2/sql?q=SELECT+gender%2C+count%28synthesys.gender%29%0A++++++++++++FROM++sanitized_data+as+synthesys%0A++++++++++++WHERE+funding_ro+like+%27%7Bfunding_round_var%7D%27%0A++++++++++++GROUP+BY+synthesys.gender
{'fields': {'count': {'type': 'number'}, 'gender': {'type': 'string'}},
 'rows': [],
 'time': 0.009,
 'total_rows': 0}


### WIDGET: DISCIPLINE (BUBBLE CHART)

This widget should only appear on the unfiltered page, or the filter by Country or filter by funding call pages.
It should not appear on the filter by discipline page. The radius of the bubbles is determined by the counts returned in the following queries. (Note that you may need to scale these values to achieve a good visual result, as you will see that these are not evenly distributed: the bubbles should show the counts on hover.)

* No filter - (again possibly embedded to save load time)
* By country
* By Synthysis funding round call

*N.B.: As you will see, not all countries have data for all disciplines. Please take care to assign the correct colors according to the UI style guide.*

In [119]:
# Discipline bubble chart: no filter
query = """SELECT synthesys.discipline, count(*)
            FROM sanitized_data AS synthesys
            WHERE synthesys.discipline IS NOT null
            GROUP BY synthesys.discipline
            ORDER BY count DESC"""
sql = {"q": query}
account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())

https://synthesys.carto.com/api/v2/sql?q=SELECT+synthesys.discipline%2C+count%28%2A%29%0A++++++++++++FROM+sanitized_data+AS+synthesys%0A++++++++++++WHERE+synthesys.discipline+IS+NOT+null%0A++++++++++++GROUP+BY+synthesys.discipline%0A++++++++++++ORDER+BY+count+DESC
{'fields': {'count': {'type': 'number'}, 'discipline': {'type': 'string'}},
 'rows': [{'count': 3238, 'discipline': 'Earth Sciences & Environment'},
          {'count': 2406, 'discipline': 'Life Sciences & Biotech'},
          {'count': 38, 'discipline': 'Humanities'},
          {'count': 15, 'discipline': 'Social Sciences'},
          {'count': 8,
           'discipline': 'Information & Communication Technologies'},
          {'count': 5, 'discipline': 'Engineering & Technology'},
          {'count': 3, 'discipline': 'Material Sciences'},
          {'count': 1, 'discipline': 'Physics'}],
 'time': 0.006,
 'total_rows': 8}


In [118]:
# Discipline bubble chart: filter by Country
iso2_var = 'ES'
query = f"""WITH gadm28 AS (SELECT the_geom_webmercator, iso2 FROM  gadm28_countries WHERE iso2 = '{iso2_var}')
             SELECT synthesys.discipline, count(*)
             FROM  sanitized_data as synthesys, gadm28
             WHERE ST_Intersects(gadm28.the_geom_webmercator, synthesys.the_geom_webmercator)
             AND synthesys.discipline IS NOT null
             GROUP BY synthesys.discipline
             ORDER BY count DESC"""

sql = {"q": query}
account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())

https://synthesys.carto.com/api/v2/sql?q=WITH+gadm28+AS+%28SELECT+the_geom_webmercator%2C+iso2+FROM++gadm28_countries+WHERE+iso2+%3D+%27ES%27%29%0A+++++++++++++SELECT+synthesys.discipline%2C+count%28%2A%29%0A+++++++++++++FROM++sanitized_data+as+synthesys%2C+gadm28%0A+++++++++++++WHERE+ST_Intersects%28gadm28.the_geom_webmercator%2C+synthesys.the_geom_webmercator%29%0A+++++++++++++AND+synthesys.discipline+IS+NOT+null%0A+++++++++++++GROUP+BY+synthesys.discipline%0A+++++++++++++ORDER+BY+count+DESC
{'fields': {'count': {'type': 'number'}, 'discipline': {'type': 'string'}},
 'rows': [{'count': 337, 'discipline': 'Earth Sciences & Environment'},
          {'count': 158, 'discipline': 'Life Sciences & Biotech'},
          {'count': 4, 'discipline': 'Humanities'}],
 'time': 0.038,
 'total_rows': 3}


In [127]:
# Discipline bubble chart: filter by funding round
funding_round_var = 'R1'
query = f"""SELECT synthesys.discipline, count(*)
            FROM sanitized_data AS synthesys
            WHERE synthesys.discipline IS NOT null
            AND synthesys.synth_roun = '{funding_round_var}'
            GROUP BY synthesys.discipline
            ORDER BY count DESC"""
sql = {"q": query}
account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())

https://synthesys.carto.com/api/v2/sql?q=SELECT+synthesys.discipline%2C+count%28%2A%29%0A++++++++++++FROM+sanitized_data+AS+synthesys%0A++++++++++++WHERE+synthesys.discipline+IS+NOT+null%0A++++++++++++AND+synthesys.synth_roun+%3D+%27R1%27%0A++++++++++++GROUP+BY+synthesys.discipline%0A++++++++++++ORDER+BY+count+DESC
{'fields': {'count': {'type': 'number'}, 'discipline': {'type': 'string'}},
 'rows': [{'count': 1563, 'discipline': 'Earth Sciences & Environment'},
          {'count': 1309, 'discipline': 'Life Sciences & Biotech'},
          {'count': 18, 'discipline': 'Humanities'},
          {'count': 8,
           'discipline': 'Information & Communication Technologies'},
          {'count': 7, 'discipline': 'Social Sciences'},
          {'count': 5, 'discipline': 'Engineering & Technology'},
          {'count': 1, 'discipline': 'Physics'}],
 'time': 0.012,
 'total_rows': 7}


## WIDGET: Researchers by time

Bar chart showing total number of researchers each year. If the y-axis value is not obvious, the chart should show the value on hover.

* No filter
* By country
* By discipline
* By funding round

*N.b. The x-axis varies across the filters (i.e. not all filters have data for all years). Indeed, some have no data at all, while others have only sparse data. The plots should be able to handle that.

In [129]:
# Time-series bar chart: no filter
query = """SELECT synthesys.start_year AS year, count(*)
            FROM sanitized_data AS synthesys
            WHERE synthesys.start_year IS NOT null
            GROUP BY synthesys.start_year
            ORDER BY synthesys.start_year ASC"""
sql = {"q": query}
account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())

https://synthesys.carto.com/api/v2/sql?q=SELECT+synthesys.start_year+AS+year%2C+count%28%2A%29%0A++++++++++++FROM+sanitized_data+AS+synthesys%0A++++++++++++WHERE+synthesys.start_year+IS+NOT+null%0A++++++++++++GROUP+BY+synthesys.start_year%0A++++++++++++ORDER+BY+synthesys.start_year+ASC
{'fields': {'count': {'type': 'number'}, 'year': {'type': 'number'}},
 'rows': [{'count': 28, 'year': 2004},
          {'count': 616, 'year': 2005},
          {'count': 656, 'year': 2006},
          {'count': 532, 'year': 2007},
          {'count': 842, 'year': 2008},
          {'count': 242, 'year': 2009},
          {'count': 268, 'year': 2010},
          {'count': 315, 'year': 2011},
          {'count': 341, 'year': 2012},
          {'count': 376, 'year': 2013},
          {'count': 342, 'year': 2014},
          {'count': 344, 'year': 2015},
          {'count': 371, 'year': 2016},
          {'count': 445, 'year': 2017}],
 'time': 0.012,
 'total_rows': 14}


In [131]:
# Time-series bar chart: filter by Country
iso2_var = 'ES'
query = f"""WITH gadm28 AS (SELECT the_geom_webmercator, iso2 FROM  gadm28_countries WHERE iso2 = '{iso2_var}')
            SELECT synthesys.start_year AS year, count(*)
            FROM sanitized_data AS synthesys, gadm28
            WHERE ST_Intersects(gadm28.the_geom_webmercator, synthesys.the_geom_webmercator)
            AND synthesys.start_year IS NOT null
            GROUP BY synthesys.start_year
            ORDER BY synthesys.start_year ASC"""
sql = {"q": query}
account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())

https://synthesys.carto.com/api/v2/sql?q=WITH+gadm28+AS+%28SELECT+the_geom_webmercator%2C+iso2+FROM++gadm28_countries+WHERE+iso2+%3D+%27ES%27%29%0A++++++++++++SELECT+synthesys.start_year+AS+year%2C+count%28%2A%29%0A++++++++++++FROM+sanitized_data+AS+synthesys%2C+gadm28%0A++++++++++++WHERE+ST_Intersects%28gadm28.the_geom_webmercator%2C+synthesys.the_geom_webmercator%29%0A++++++++++++AND+synthesys.start_year+IS+NOT+null%0A++++++++++++GROUP+BY+synthesys.start_year%0A++++++++++++ORDER+BY+synthesys.start_year+ASC
{'fields': {'count': {'type': 'number'}, 'year': {'type': 'number'}},
 'rows': [{'count': 2, 'year': 2004},
          {'count': 30, 'year': 2005},
          {'count': 47, 'year': 2006},
          {'count': 34, 'year': 2007},
          {'count': 62, 'year': 2008},
          {'count': 25, 'year': 2009},
          {'count': 36, 'year': 2010},
          {'count': 28, 'year': 2011},
          {'count': 35, 'year': 2012},
          {'count': 32, 'year': 2013},
          {'count': 42, 'ye

In [132]:
# Time-series bar chart: filter by discipline
discipline_var = 'Earth Sciences & Environment'
query = f"""SELECT synthesys.start_year AS year, count(*)
            FROM sanitized_data AS synthesys
            WHERE synthesys.start_year IS NOT null
            AND synthesys.discipline = '{discipline_var}'
            GROUP BY synthesys.start_year
            ORDER BY synthesys.start_year ASC"""
sql = {"q": query}
account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())

https://synthesys.carto.com/api/v2/sql?q=SELECT+synthesys.start_year+AS+year%2C+count%28%2A%29%0A++++++++++++FROM+sanitized_data+AS+synthesys%0A++++++++++++WHERE+synthesys.start_year+IS+NOT+null%0A++++++++++++AND+synthesys.discipline+%3D+%27Earth+Sciences+%26+Environment%27%0A++++++++++++GROUP+BY+synthesys.start_year%0A++++++++++++ORDER+BY+synthesys.start_year+ASC
{'fields': {'count': {'type': 'number'}, 'year': {'type': 'number'}},
 'rows': [{'count': 12, 'year': 2004},
          {'count': 296, 'year': 2005},
          {'count': 369, 'year': 2006},
          {'count': 276, 'year': 2007},
          {'count': 489, 'year': 2008},
          {'count': 122, 'year': 2009},
          {'count': 152, 'year': 2010},
          {'count': 195, 'year': 2011},
          {'count': 218, 'year': 2012},
          {'count': 249, 'year': 2013},
          {'count': 221, 'year': 2014},
          {'count': 185, 'year': 2015},
          {'count': 193, 'year': 2016},
          {'count': 261, 'year': 2017}],
 't

In [134]:
# Time-series bar chart: filter by funding round
funding_round_var = 'R1'
query = f"""SELECT synthesys.start_year AS year, count(*)
            FROM sanitized_data AS synthesys
            WHERE synthesys.start_year IS NOT null
            AND synthesys.synth_roun = '{funding_round_var}'
            GROUP BY synthesys.start_year
            ORDER BY synthesys.start_year ASC"""
sql = {"q": query}
account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())

https://synthesys.carto.com/api/v2/sql?q=SELECT+synthesys.start_year+AS+year%2C+count%28%2A%29%0A++++++++++++FROM+sanitized_data+AS+synthesys%0A++++++++++++WHERE+synthesys.start_year+IS+NOT+null%0A++++++++++++AND+synthesys.synth_roun+%3D+%27R1%27%0A++++++++++++GROUP+BY+synthesys.start_year%0A++++++++++++ORDER+BY+synthesys.start_year+ASC
{'fields': {'count': {'type': 'number'}, 'year': {'type': 'number'}},
 'rows': [{'count': 28, 'year': 2004},
          {'count': 616, 'year': 2005},
          {'count': 656, 'year': 2006},
          {'count': 532, 'year': 2007},
          {'count': 842, 'year': 2008},
          {'count': 241, 'year': 2009}],
 'time': 0.007,
 'total_rows': 6}


## WIDGET: RESEARCHER TYPE

Researcher type denotes the academic level of the person. These are defined by five keys (EXP, PDOC, PGR, TEC, and UND). The full english name of these positions is mapped to these keys in the below json (and should be used to generate labels on the donut plot).

```json
{'EXP': 'Experienced Researcher',
 'PDOC': 'Postdoctoral Researcher',
 'PGR': 'Postgraduate Researcher',
 'TEC': 'Technician',
 'UND': 'Undergraduate'}
```

The donut plot should appear on all pages of the dashboard.  Percentages and labels should be shown on hover.

* No filter
* By country
* By discipline
* By funding round

In [144]:
# Donut plot for researcher type: no filter

query = """SELECT synthesys.researcher AS year, count(*)
            FROM sanitized_data AS synthesys
            WHERE synthesys.researcher IS NOT null
            GROUP BY synthesys.researcher
            ORDER BY synthesys.count DESC"""
sql = {"q": query}
account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())

https://synthesys.carto.com/api/v2/sql?q=SELECT+synthesys.researcher+AS+year%2C+count%28%2A%29%0A++++++++++++FROM+sanitized_data+AS+synthesys%0A++++++++++++WHERE+synthesys.researcher+IS+NOT+null%0A++++++++++++GROUP+BY+synthesys.researcher%0A++++++++++++ORDER+BY+synthesys.count+DESC
{'fields': {'count': {'type': 'number'}, 'year': {'type': 'string'}},
 'rows': [{'count': 1914, 'year': 'EXP'},
          {'count': 1892, 'year': 'PDOC'},
          {'count': 1733, 'year': 'PGR'},
          {'count': 124, 'year': 'UND'},
          {'count': 55, 'year': 'TEC'}],
 'time': 0.006,
 'total_rows': 5}


In [147]:
# Donut plot for researcher type: filter by country
iso2_var = 'ES'
query = f"""WITH gadm28 AS (SELECT the_geom_webmercator, iso2 FROM  gadm28_countries WHERE iso2 = '{iso2_var}')
            SELECT synthesys.researcher AS year, count(*)
            FROM sanitized_data AS synthesys, gadm28
            WHERE ST_Intersects(gadm28.the_geom_webmercator, synthesys.the_geom_webmercator)
            AND synthesys.researcher IS NOT null
            GROUP BY synthesys.researcher
            ORDER BY synthesys.count DESC"""
sql = {"q": query}
account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())

https://synthesys.carto.com/api/v2/sql?q=WITH+gadm28+AS+%28SELECT+the_geom_webmercator%2C+iso2+FROM++gadm28_countries+WHERE+iso2+%3D+%27ES%27%29%0A++++++++++++SELECT+synthesys.researcher+AS+year%2C+count%28%2A%29%0A++++++++++++FROM+sanitized_data+AS+synthesys%2C+gadm28%0A++++++++++++WHERE+ST_Intersects%28gadm28.the_geom_webmercator%2C+synthesys.the_geom_webmercator%29%0A++++++++++++AND+synthesys.researcher+IS+NOT+null%0A++++++++++++GROUP+BY+synthesys.researcher%0A++++++++++++ORDER+BY+synthesys.count+DESC
{'fields': {'count': {'type': 'number'}, 'year': {'type': 'string'}},
 'rows': [{'count': 191, 'year': 'PGR'},
          {'count': 182, 'year': 'PDOC'},
          {'count': 126, 'year': 'EXP'},
          {'count': 1, 'year': 'TEC'}],
 'time': 0.029,
 'total_rows': 4}


In [148]:
# Donut plot for researcher type: filter by discipline
discipline_var = 'Earth Sciences & Environment'
query = f"""SELECT synthesys.researcher, count(*)
            FROM sanitized_data AS synthesys
            WHERE synthesys.researcher IS NOT null
            AND synthesys.discipline = '{discipline_var}'
            GROUP BY synthesys.researcher
            ORDER BY synthesys.count ASC"""
sql = {"q": query}
account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())

https://synthesys.carto.com/api/v2/sql?q=SELECT+synthesys.researcher%2C+count%28%2A%29%0A++++++++++++FROM+sanitized_data+AS+synthesys%0A++++++++++++WHERE+synthesys.researcher+IS+NOT+null%0A++++++++++++AND+synthesys.discipline+%3D+%27Earth+Sciences+%26+Environment%27%0A++++++++++++GROUP+BY+synthesys.researcher%0A++++++++++++ORDER+BY+synthesys.count+ASC
{'fields': {'count': {'type': 'number'}, 'researcher': {'type': 'string'}},
 'rows': [{'count': 37, 'researcher': 'TEC'},
          {'count': 58, 'researcher': 'UND'},
          {'count': 1005, 'researcher': 'EXP'},
          {'count': 1013, 'researcher': 'PGR'},
          {'count': 1125, 'researcher': 'PDOC'}],
 'time': 0.01,
 'total_rows': 5}


In [191]:
# Donut plot for researcher type: filter by funding round
funding_round_var = 'R1'
query = f"""SELECT synthesys.researcher, count(*)
            FROM sanitized_data AS synthesys
            WHERE synthesys.researcher IS NOT null
            AND synthesys.funding_ro = '{funding_round_var}'
            GROUP BY synthesys.researcher
            ORDER BY synthesys.count ASC"""
sql = {"q": query}
account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())

https://synthesys.carto.com/api/v2/sql?q=SELECT+synthesys.researcher%2C+count%28%2A%29%0A++++++++++++FROM+sanitized_data+AS+synthesys%0A++++++++++++WHERE+synthesys.researcher+IS+NOT+null%0A++++++++++++AND+synthesys.funding_ro+%3D+%27R1%27%0A++++++++++++GROUP+BY+synthesys.researcher%0A++++++++++++ORDER+BY+synthesys.count+ASC
{'error': ['cannot connect to the database']}


## Widgets: Institute visited

This should be a [Treemaps](https://en.wikipedia.org/wiki/Treemapping) showing where the researchers go to on their visits (which specific institutes). The size of each box should relate to the counts. The counts and institute name should be viewable on hover. Colours should be fixed according to the style guide.
```json
{0: 'Botanischer Garten und Botanisches Museum Collections',
 1: 'Botanischer Garten und Botanisches Museum Herbarium',
 2: 'Botanischer Garten und Botanisches Museum Laboratories',
 3: 'Botanischer Garten und Botanisches Museum Living Collections',
 4: 'Botanischer Garten und Botanisches Museum Molecular Laboratory',
 5: 'Botanischer Garten und Botanisches Museum Scanning Electron Microscope facilities',
 6: 'Consejo Superior de Investigaciones Científicas',
 7: 'Hungarian Natural History Museum',
 8: 'Hungarian Natural History Museum Collections',
 9: 'Hungarian Natural History Museum Computing facilities',
 10: 'Hungarian Natural History Museum Microscopy facilities',
 11: 'Hungarian Natural History Museum Molecular Analysis and Microscopy facilities',
 12: 'Museum National d’Histoire Naturelle Analytical Facilities',
 13: 'Museum National d’Histoire Naturelle Collections',
 14: 'Museum National d’Histoire Naturelle Facilities',
 15: 'Museum National d’Histoire Naturelle Mass Spectrometry facilities',
 16: 'Museum National d’Histoire Naturelle Molecular Laboratory',
 17: 'Museum National d’Histoire Naturelle Scanning Electron Microscope facilities',
 18: 'Museum and Institute of Zoology of the Polish Academy of Sciences',
 19: 'Museum für Naturkunde Collections',
 20: 'Museum für Naturkunde Laboratories',
 21: 'Museum für Naturkunde Mineralogy',
 22: 'Museum für Naturkunde Palaeontology',
 23: 'Museum für Naturkunde Zoology',
 24: 'National Museum Prague',
 25: 'Natural History Museum London Analytical Facilities',
 26: 'Natural History Museum London Collections',
 27: 'Natural History Museum London Collections and Laboratories',
 28: 'Natural History Museum London Laboratories',
 29: 'Natural History Museum Vienna',
 30: 'Naturalis Biodiversity Center',
 31: 'Naturhistoriska riksmuseet',
 32: 'Royal Belgian Institute of Natural Sciences',
 33: 'Royal Botanic Garden Edinburgh',
 34: 'Royal Botanic Gardens Kew',
 35: 'Royal Museum of Central Africa',
 36: 'Senckenberg Gesellschaft für Naturforschung Collections',
 37: 'Senckenberg Gesellschaft für Naturforschung Laboratories',
 38: 'State Museum of Natural History Stuttgart',
 39: 'University of Amsterdam',
 40: 'University of Copenhagen'}
```

Note: There are 41 unique institute names. The mapping of institute ID to human readable strings is given in JSON above). Also, I think that you can decide, depending on your best judgement regarding how the figure looks to aggregate institutes with few visits (into an 'Other') category.

This widget should appear on all dashboard pages:
* No filter
* By country
* By discipline
* By funding round

In [3]:
# Treemap widget for collection visited type: no filter

query = """SELECT synthesys.inst_id AS institute_id, count(*)
            FROM sanitized_data AS synthesys
            WHERE synthesys.inst_short IS NOT null
            GROUP BY synthesys.inst_id
            ORDER BY synthesys.count DESC"""
sql = {"q": query}
account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())
# At this point you can use the json mapping to convert the
# institute_id to an english name to appear on the widget

https://synthesys.carto.com/api/v2/sql?q=SELECT+synthesys.inst_id+AS+institute_id%2C+count%28%2A%29%0A++++++++++++FROM+sanitized_data+AS+synthesys%0A++++++++++++WHERE+synthesys.inst_short+IS+NOT+null%0A++++++++++++GROUP+BY+synthesys.inst_id%0A++++++++++++ORDER+BY+synthesys.count+DESC
{'fields': {'count': {'type': 'number'}, 'institute_id': {'type': 'number'}},
 'rows': [{'count': 484, 'institute_id': 13},
          {'count': 413, 'institute_id': 26},
          {'count': 408, 'institute_id': 27},
          {'count': 390, 'institute_id': 29},
          {'count': 361, 'institute_id': 40},
          {'count': 338, 'institute_id': 31},
          {'count': 311, 'institute_id': 6},
          {'count': 299, 'institute_id': 25},
          {'count': 279, 'institute_id': 30},
          {'count': 229, 'institute_id': 32},
          {'count': 207, 'institute_id': 8},
          {'count': 165, 'institute_id': 34},
          {'count': 154, 'institute_id': 35},
          {'count': 151, 'institute_id': 

In [4]:
# Treemap widget for collection visited type: filter by country
iso2_var = 'ES'
query = f"""WITH gadm28 AS (SELECT the_geom_webmercator, iso2 FROM  gadm28_countries WHERE iso2 = '{iso2_var}')
            SELECT synthesys.inst_id AS institute_id, count(*)
            FROM sanitized_data AS synthesys, gadm28
            WHERE ST_Intersects(gadm28.the_geom_webmercator, synthesys.the_geom_webmercator)
            AND synthesys.inst_short IS NOT null
            GROUP BY synthesys.inst_id
            ORDER BY synthesys.count DESC"""
sql = {"q": query}
account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())

https://synthesys.carto.com/api/v2/sql?q=WITH+gadm28+AS+%28SELECT+the_geom_webmercator%2C+iso2+FROM++gadm28_countries+WHERE+iso2+%3D+%27ES%27%29%0A++++++++++++SELECT+synthesys.inst_id+AS+institute_id%2C+count%28%2A%29%0A++++++++++++FROM+sanitized_data+AS+synthesys%2C+gadm28%0A++++++++++++WHERE+ST_Intersects%28gadm28.the_geom_webmercator%2C+synthesys.the_geom_webmercator%29%0A++++++++++++AND+synthesys.inst_short+IS+NOT+null%0A++++++++++++GROUP+BY+synthesys.inst_id%0A++++++++++++ORDER+BY+synthesys.count+DESC
{'fields': {'count': {'type': 'number'}, 'institute_id': {'type': 'number'}},
 'rows': [{'count': 60, 'institute_id': 13},
          {'count': 52, 'institute_id': 27},
          {'count': 38, 'institute_id': 26},
          {'count': 30, 'institute_id': 30},
          {'count': 29, 'institute_id': 29},
          {'count': 28, 'institute_id': 25},
          {'count': 27, 'institute_id': 40},
          {'count': 23, 'institute_id': 14},
          {'count': 23, 'institute_id': 31},
     

In [5]:
# Treemap widget for collection visited type: filter by discipline
discipline_var = 'Humanities'
query = f"""SELECT synthesys.inst_id AS institute_id, count(*)
            FROM sanitized_data AS synthesys
            WHERE synthesys.inst_short IS NOT null
            AND synthesys.discipline = '{discipline_var}'
            GROUP BY synthesys.inst_id
            ORDER BY synthesys.count DESC"""
sql = {"q": query}
account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())

https://synthesys.carto.com/api/v2/sql?q=SELECT+synthesys.inst_id+AS+institute_id%2C+count%28%2A%29%0A++++++++++++FROM+sanitized_data+AS+synthesys%0A++++++++++++WHERE+synthesys.inst_short+IS+NOT+null%0A++++++++++++AND+synthesys.discipline+%3D+%27Humanities%27%0A++++++++++++GROUP+BY+synthesys.inst_id%0A++++++++++++ORDER+BY+synthesys.count+DESC
{'fields': {'count': {'type': 'number'}, 'institute_id': {'type': 'number'}},
 'rows': [{'count': 6, 'institute_id': 13},
          {'count': 6, 'institute_id': 40},
          {'count': 5, 'institute_id': 6},
          {'count': 5, 'institute_id': 29},
          {'count': 2, 'institute_id': 32},
          {'count': 2, 'institute_id': 27},
          {'count': 2, 'institute_id': 8},
          {'count': 1, 'institute_id': 26},
          {'count': 1, 'institute_id': 11},
          {'count': 1, 'institute_id': 38},
          {'count': 1, 'institute_id': 30},
          {'count': 1, 'institute_id': 15},
          {'count': 1, 'institute_id': 16},
       

In [7]:
# Treemap widget for collection visited type: filter by funding round
synth_round_var = 'R1'
query = f"""SELECT synthesys.inst_id AS institute_id, count(*)
            FROM sanitized_data AS synthesys
            WHERE synthesys.inst_short IS NOT null
            AND synthesys.synth_roun = '{synth_round_var}'
            GROUP BY synthesys.inst_id
            ORDER BY synthesys.count DESC"""
sql = {"q": query}
account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())

https://synthesys.carto.com/api/v2/sql?q=SELECT+synthesys.inst_id+AS+institute_id%2C+count%28%2A%29%0A++++++++++++FROM+sanitized_data+AS+synthesys%0A++++++++++++WHERE+synthesys.inst_short+IS+NOT+null%0A++++++++++++AND+synthesys.synth_roun+%3D+%27R1%27%0A++++++++++++GROUP+BY+synthesys.inst_id%0A++++++++++++ORDER+BY+synthesys.count+DESC
{'fields': {'count': {'type': 'number'}, 'institute_id': {'type': 'number'}},
 'rows': [{'count': 413, 'institute_id': 26},
          {'count': 203, 'institute_id': 13},
          {'count': 200, 'institute_id': 40},
          {'count': 193, 'institute_id': 29},
          {'count': 190, 'institute_id': 31},
          {'count': 160, 'institute_id': 30},
          {'count': 156, 'institute_id': 6},
          {'count': 126, 'institute_id': 25},
          {'count': 102, 'institute_id': 32},
          {'count': 94, 'institute_id': 23},
          {'count': 81, 'institute_id': 28},
          {'count': 81, 'institute_id': 34},
          {'count': 76, 'institute_id

## DYNAMIC SENTENCES

Note, depending on the filter page, a different sentence will be needed. You will also need metadata json to construct these in some cases (all provided here).


In [170]:
# No filter (should appear immediatley on the dashboard page, and the json can be embedded for speed)

query = """SELECT COUNT(DISTINCT(synthesys.home_insti)) as institutes,
            COUNT(synthesys.visit_days) AS total_visitors,
            SUM(synthesys.visit_days) AS days
            FROM sanitized_data AS synthesys
            WHERE synthesys.visit_days IS NOT null"""
sql = {"q": query}
account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())
r.json()


days = r.json().get('rows')[0].get('days')
institutes = r.json().get('rows')[0].get('institutes')
total_visitors = r.json().get('rows')[0].get('total_visitors')

print(f"\n {} {total_visitors:,g} vistors, from {institutes:,g} institutes, were granted a total of {days:,g} research days.")

https://synthesys.carto.com/api/v2/sql?q=SELECT+COUNT%28DISTINCT%28synthesys.home_insti%29%29+as+institutes%2C%0A++++++++++++COUNT%28synthesys.visit_days%29+AS+total_visitors%2C%0A++++++++++++SUM%28synthesys.visit_days%29+AS+days%0A++++++++++++FROM+sanitized_data+AS+synthesys%0A++++++++++++WHERE+synthesys.visit_days+IS+NOT+null
{'fields': {'days': {'type': 'number'},
            'institutes': {'type': 'number'},
            'total_visitors': {'type': 'number'}},
 'rows': [{'days': 69472, 'institutes': 767, 'total_visitors': 5718}],
 'time': 0.009,
 'total_rows': 1}

5,718 vistors, from 767 institutes, were granted a total of 69,472 research days.


In [174]:
iso2_dic = {'AL': 'Albania', 'AT': 'Austria', 'BA': 'Bosnia and Herzegovina', 'BE': 'Belgium', 'BG': 'Bulgaria', 'CH': 'Switzerland', 'CY': 'Cyprus', 'CZ': 'Czech Republic', 'DE': 'Germany', 'DK': 'Denmark', 'EE': 'Estonia', 'ES': 'Spain', 'FI': 'Finland', 'FR': 'France', 'GB': 'United Kingdom', 'GF': 'French Guiana', 'GR': 'Greece', 'HR': 'Croatia', 'HU': 'Hungary', 'IE': 'Ireland', 'IL': 'Israel', 'IS': 'Iceland', 'IT': 'Italy', 'LI': 'Liechtenstein', 'LT': 'Lithuania', 'LU': 'Luxembourg', 'LV': 'Latvia', 'MK': 'Macedonia', 'MT': 'Malta', 'NL': 'Netherlands', 'NO': 'Norway', 'PL': 'Poland', 'PS': 'Palestina', 'PT': 'Portugal', 'RE': 'Reunion', 'RO': 'Romania', 'RS': 'Serbia', 'SE': 'Sweden', 'SI': 'Slovenia', 'SK': 'Slovakia', 'TR': 'Turkey'}

In [176]:
# Filter by country 

iso2_var = 'ES'
query = f"""WITH gadm28 AS (SELECT the_geom_webmercator, iso2 FROM  gadm28_countries WHERE iso2 = '{iso2_var}')
            SELECT COUNT(DISTINCT(synthesys.home_insti)) as institutes,
            COUNT(synthesys.visit_days) AS total_visitors,
            SUM(synthesys.visit_days) AS days
            FROM sanitized_data AS synthesys, gadm28
            WHERE ST_Intersects(gadm28.the_geom_webmercator, synthesys.the_geom_webmercator)
            AND synthesys.visit_days IS NOT null"""
sql = {"q": query}
account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())
r.json()

country = iso2_dic[iso2_var]
days = r.json().get('rows')[0].get('days')
institutes = r.json().get('rows')[0].get('institutes')
total_visitors = r.json().get('rows')[0].get('total_visitors')

print(f"\n{country} had {total_visitors:,g} vistors, from {institutes:,g} institutes, and was granted a total of {days:,g} research days.")

https://synthesys.carto.com/api/v2/sql?q=WITH+gadm28+AS+%28SELECT+the_geom_webmercator%2C+iso2+FROM++gadm28_countries+WHERE+iso2+%3D+%27ES%27%29%0A++++++++++++SELECT+COUNT%28DISTINCT%28synthesys.home_insti%29%29+as+institutes%2C%0A++++++++++++COUNT%28synthesys.visit_days%29+AS+total_visitors%2C%0A++++++++++++SUM%28synthesys.visit_days%29+AS+days%0A++++++++++++FROM+sanitized_data+AS+synthesys%2C+gadm28%0A++++++++++++WHERE+ST_Intersects%28gadm28.the_geom_webmercator%2C+synthesys.the_geom_webmercator%29%0A++++++++++++AND+synthesys.visit_days+IS+NOT+null
{'fields': {'days': {'type': 'number'},
            'institutes': {'type': 'number'},
            'total_visitors': {'type': 'number'}},
 'rows': [{'days': 6315, 'institutes': 87, 'total_visitors': 500}],
 'time': 0.076,
 'total_rows': 1}

Spain had 500 vistors, from 87 institutes, and was granted a total of 6,315 research days.


In [182]:
# Filter by discipline

discipline_var = 'Humanities'
query = f"""SELECT COUNT(DISTINCT(synthesys.home_insti)) as institutes,
            COUNT(synthesys.visit_days) AS total_visitors,
            SUM(synthesys.visit_days) AS days
            FROM sanitized_data AS synthesys
            WHERE synthesys.visit_days IS NOT null
            AND synthesys.discipline = '{discipline_var}'
            """
sql = {"q": query}
account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())
r.json()


days = r.json().get('rows')[0].get('days')
institutes = r.json().get('rows')[0].get('institutes')
total_visitors = r.json().get('rows')[0].get('total_visitors')

print(f"""\n A total of {total_visitors:,g} vistors related to {discipline_var} disciplines came from {institutes:,g} institutes, and were granted a total of {days:,g} research days.""")

https://synthesys.carto.com/api/v2/sql?q=SELECT+COUNT%28DISTINCT%28synthesys.home_insti%29%29+as+institutes%2C%0A++++++++++++COUNT%28synthesys.visit_days%29+AS+total_visitors%2C%0A++++++++++++SUM%28synthesys.visit_days%29+AS+days%0A++++++++++++FROM+sanitized_data+AS+synthesys%0A++++++++++++WHERE+synthesys.visit_days+IS+NOT+null%0A++++++++++++AND+synthesys.discipline+%3D+%27Humanities%27%0A++++++++++++
{'fields': {'days': {'type': 'number'},
            'institutes': {'type': 'number'},
            'total_visitors': {'type': 'number'}},
 'rows': [{'days': 478, 'institutes': 26, 'total_visitors': 38}],
 'time': 0.012,
 'total_rows': 1}

 A total of 38 vistors related to Humanities disciplines came from 26 institutes, and were granted a total of 478 research days.


In [189]:
funding_rounds = {"R1":"SYNTHESYS (2004 - 2009)", "R2":"SYNTHESYS2 (2009 - 2013)","R3":"SYNTHESYS3 (2013 - 2017)"}

In [190]:
# Filter by funding round

funding_round_var = 'R3'
query = f"""SELECT COUNT(DISTINCT(synthesys.home_insti)) as institutes,
            COUNT(synthesys.visit_days) AS total_visitors,
            SUM(synthesys.visit_days) AS days
            FROM sanitized_data AS synthesys
            WHERE synthesys.visit_days IS NOT null
            AND synthesys.synth_roun = '{funding_round_var}'
            """
sql = {"q": query}
account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())
r.json()

funding_round = funding_rounds[funding_round_var]
days = r.json().get('rows')[0].get('days')
institutes = r.json().get('rows')[0].get('institutes')
total_visitors = r.json().get('rows')[0].get('total_visitors')

print(f"""\n A total of {total_visitors:,g} vistors came in {funding_round}, from {institutes:,g} institutes, and were granted a total of {days:,g} research days.""")

https://synthesys.carto.com/api/v2/sql?q=SELECT+COUNT%28DISTINCT%28synthesys.home_insti%29%29+as+institutes%2C%0A++++++++++++COUNT%28synthesys.visit_days%29+AS+total_visitors%2C%0A++++++++++++SUM%28synthesys.visit_days%29+AS+days%0A++++++++++++FROM+sanitized_data+AS+synthesys%0A++++++++++++WHERE+synthesys.visit_days+IS+NOT+null%0A++++++++++++AND+synthesys.synth_roun+%3D+%27R3%27%0A++++++++++++
{'fields': {'days': {'type': 'number'},
            'institutes': {'type': 'number'},
            'total_visitors': {'type': 'number'}},
 'rows': [{'days': 15388, 'institutes': 376, 'total_visitors': 1507}],
 'time': 0.008,
 'total_rows': 1}

 A total of 1,507 vistors came in SYNTHESYS3 (2013 - 2017), from 376 institutes, and were granted a total of 15,388 research days.


## Sankey plot

We will need to add a Sankey diagram as a widget on the Dashboard. **This should only appear when a user has filtered by COUNTRY**.

We are going to use the D3-like example for clarity from [this python repo](https://github.com/ricklupton/ipysankeywidget) to demonstrate how to create sankeys. However, we do not have any prefrence for the library you use to implement this. This is just as a simple demo, showing how to go from the Carto database query to the two variants of sankey we wish to create.


If you want to run the cells in this notebook to test you will need to run:
* `pip install ipysankeywidget`

* `jupyter nbextension enable --py --sys-prefix ipysankeywidget`

In [3]:
from ipysankeywidget import SankeyWidget
from ipywidgets import Layout
import json
import requests

In [5]:
# We require metadata to build the sankeys. 

institute_lookups = {0: 'Botanischer Garten und Botanisches Museum Collections',
 1: 'Botanischer Garten und Botanisches Museum Herbarium',
 2: 'Botanischer Garten und Botanisches Museum Laboratories',
 3: 'Botanischer Garten und Botanisches Museum Living Collections',
 4: 'Botanischer Garten und Botanisches Museum Molecular Laboratory',
 5: 'Botanischer Garten und Botanisches Museum Scanning Electron Microscope facilities',
 6: 'Consejo Superior de Investigaciones Científicas',
 7: 'Hungarian Natural History Museum',
 8: 'Hungarian Natural History Museum Collections',
 9: 'Hungarian Natural History Museum Computing facilities',
 10: 'Hungarian Natural History Museum Microscopy facilities',
 11: 'Hungarian Natural History Museum Molecular Analysis and Microscopy facilities',
 12: 'Museum National d’Histoire Naturelle Analytical Facilities',
 13: 'Museum National d’Histoire Naturelle Collections',
 14: 'Museum National d’Histoire Naturelle Facilities',
 15: 'Museum National d’Histoire Naturelle Mass Spectrometry facilities',
 16: 'Museum National d’Histoire Naturelle Molecular Laboratory',
 17: 'Museum National d’Histoire Naturelle Scanning Electron Microscope facilities',
 18: 'Museum and Institute of Zoology of the Polish Academy of Sciences',
 19: 'Museum für Naturkunde Collections',
 20: 'Museum für Naturkunde Laboratories',
 21: 'Museum für Naturkunde Mineralogy',
 22: 'Museum für Naturkunde Palaeontology',
 23: 'Museum für Naturkunde Zoology',
 24: 'National Museum Prague',
 25: 'Natural History Museum London Analytical Facilities',
 26: 'Natural History Museum London Collections',
 27: 'Natural History Museum London Collections and Laboratories',
 28: 'Natural History Museum London Laboratories',
 29: 'Natural History Museum Vienna',
 30: 'Naturalis Biodiversity Center',
 31: 'Naturhistoriska riksmuseet',
 32: 'Royal Belgian Institute of Natural Sciences',
 33: 'Royal Botanic Garden Edinburgh',
 34: 'Royal Botanic Gardens Kew',
 35: 'Royal Museum of Central Africa',
 36: 'Senckenberg Gesellschaft für Naturforschung Collections',
 37: 'Senckenberg Gesellschaft für Naturforschung Laboratories',
 38: 'State Museum of Natural History Stuttgart',
 39: 'University of Amsterdam',
 40: 'University of Copenhagen'}

iso_to_country = {'ALB': 'Albania',
 'AUT': 'Austria',
 'BEL': 'Belgium',
 'BGR': 'Bulgaria',
 'BIH': 'Bosnia and Herzegovina',
 'CHE': 'Switzerland',
 'CYP': 'Cyprus',
 'CZE': 'Czech Republic',
 'DEU': 'Germany',
 'DNK': 'Denmark',
 'ESP': 'Spain',
 'EST': 'Estonia',
 'FIN': 'Finland',
 'FRA': 'France',
 'GBR': 'United Kingdom',
 'GRC': 'Greece',
 'GUF': 'French Guiana',
 'HRV': 'Croatia',
 'HUN': 'Hungary',
 'IRL': 'Ireland',
 'ISL': 'Iceland',
 'ISR': 'Israel',
 'ITA': 'Italy',
 'LIE': 'Liechtenstein',
 'LTU': 'Lithuania',
 'LUX': 'Luxembourg',
 'LVA': 'Latvia',
 'MKD': 'Macedonia',
 'NLD': 'Netherlands',
 'NOR': 'Norway',
 'POL': 'Poland',
 'PRT': 'Portugal',
 'PSE': 'Palestina',
 'REU': 'Reunion',
 'ROU': 'Romania',
 'SRB': 'Serbia',
 'SVK': 'Slovakia',
 'SVN': 'Slovenia',
 'SWE': 'Sweden',
 'TUR': 'Turkey'}

In [17]:
country = "GBR"

query =f""" with gadm28 as (SELECT the_geom_webmercator, iso, name_1 FROM  gadm28_adm1 WHERE iso = '{country}')
 SELECT gadm28.name_1, COUNT(synthesys.home_insti) as counts, synthesys.inst_id
 FROM  sanitized_data as synthesys, gadm28
 where ST_Intersects(  gadm28.the_geom_webmercator, synthesys.the_geom_webmercator)
 GROUP BY gadm28.name_1, inst_id
 ORDER BY inst_id"""

account = 'synthesys'
urlCarto = f"https://{account}.carto.com/api/v2/sql"

sql = {"q": query}
r = requests.get(urlCarto, params=sql)
print(r.url)

https://synthesys.carto.com/api/v2/sql?q=+with+gadm28+as+%28SELECT+the_geom_webmercator%2C+iso%2C+name_1+FROM++gadm28_adm1+WHERE+iso+%3D+%27GBR%27%29%0A+SELECT+gadm28.name_1%2C+COUNT%28synthesys.home_insti%29+as+counts%2C+synthesys.inst_id%0A+FROM++sanitized_data+as+synthesys%2C+gadm28%0A+where+ST_Intersects%28++gadm28.the_geom_webmercator%2C+synthesys.the_geom_webmercator%29%0A+GROUP+BY+gadm28.name_1%2C+inst_id%0A+ORDER+BY+inst_id


In [18]:
# We will need to create a array that holds a json object, mapping the nodes: [{source, target, value}] 
links = []
for row in r.json().get('rows'):
    links.append({'source':row.get('name_1'),
                  'value':row.get('counts'),
                  'target':institute_lookups[row.get('inst_id')]})

In [19]:
def sankey(width=1000, height=1000, **value, ):
    """Show SankeyWidget with default values for size and margins"""
    layout = Layout(width=str(width), height=str(height))
    return SankeyWidget(layout=layout,
                        margins=dict(top=10, bottom=0, left=150, right=300),
                        **value)

In [20]:
s = sankey(links=links)
s